In [1]:
#Load Packages
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import re
from urllib.request import Request, urlopen

In [2]:
base_url = 'https://footystats.org/england/championship'
team = str()
url = base_url + team

In [3]:
#Load Site
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

req = Request("https://footystats.org/england/championship", headers=headers)
html_page = urlopen(req)

In [4]:
print(html_page)

In [5]:
soup = BeautifulSoup(html_page, "lxml")

In [6]:
links = []
for link in soup.findAll('a', class_="bold hover-modal-parent hover-modal-ajax-team"):
    links.append(link.get('href'))

In [7]:
print(links)

['/clubs/afc-bournemouth-148', '/clubs/west-bromwich-albion-fc-142', '/clubs/coventry-city-fc-239', '/clubs/fulham-fc-162', '/clubs/stoke-city-fc-141', '/clubs/blackburn-rovers-fc-216', '/clubs/bristol-city-fc-207', '/clubs/huddersfield-town-fc-217', '/clubs/reading-fc-219', '/clubs/queens-park-rangers-fc-160', '/clubs/sheffield-united-fc-251', '/clubs/birmingham-city-fc-206', '/clubs/cardiff-city-fc-161', '/clubs/blackpool-fc-228', '/clubs/preston-north-end-fc-210', '/clubs/swansea-city-afc-154', '/clubs/luton-town-fc-271', '/clubs/middlesbrough-fc-147', '/clubs/millwall-fc-229', '/clubs/barnsley-fc-215', '/clubs/peterborough-united-fc-250', '/clubs/nottingham-forest-fc-211', '/clubs/hull-city-afc-150', '/clubs/derby-county-fc-213']


In [8]:
#Create an empty list for us to send each team's link to
teamLinks = []

In [10]:
#For each link...
for i in range(0,24):
    
    #...Find the page the link is going to...
    temp = links[i]
    
    #...Add the link to the website domain...
    temp = "https://footystats.org" + temp
    
    #...Add the finished link to our teamLinks list...
    teamLinks.append(temp)

In [11]:
teamLinks

['https://footystats.org/clubs/afc-bournemouth-148',
 'https://footystats.org/clubs/west-bromwich-albion-fc-142',
 'https://footystats.org/clubs/coventry-city-fc-239',
 'https://footystats.org/clubs/fulham-fc-162',
 'https://footystats.org/clubs/stoke-city-fc-141',
 'https://footystats.org/clubs/blackburn-rovers-fc-216',
 'https://footystats.org/clubs/bristol-city-fc-207',
 'https://footystats.org/clubs/huddersfield-town-fc-217',
 'https://footystats.org/clubs/reading-fc-219',
 'https://footystats.org/clubs/queens-park-rangers-fc-160',
 'https://footystats.org/clubs/sheffield-united-fc-251',
 'https://footystats.org/clubs/birmingham-city-fc-206',
 'https://footystats.org/clubs/cardiff-city-fc-161',
 'https://footystats.org/clubs/blackpool-fc-228',
 'https://footystats.org/clubs/preston-north-end-fc-210',
 'https://footystats.org/clubs/swansea-city-afc-154',
 'https://footystats.org/clubs/luton-town-fc-271',
 'https://footystats.org/clubs/middlesbrough-fc-147',
 'https://footystats.org/

In [12]:
#Create empty lists for players
playerLinks = []
plinks = []

In [13]:
#For each team link page...
for i in range(len(teamLinks)):

    reqs = Request(teamLinks[i], headers=headers)
    html_page = urlopen(reqs)
    
    psoup = BeautifulSoup(html_page, "lxml")
    
    for link in psoup.findAll('a', class_="semi-bold"):
            plinks.append(link.get('href'))

In [14]:
#remove links that do not relate to players
plinks = [i for i in plinks if i and i.startswith('/players/')]

In [15]:
#remove duplicates
plinks = list(set(plinks))

In [17]:
#For each plink...
for i in range(len(plinks)):
    
    #...Find the page the link is going to...
    ptemp = plinks[i]
    
    #...Add the link to the website domain...
    ptemp = "https://footystats.org" + ptemp
    
    #...Add the finished link to our playerLinks list...
    playerLinks.append(ptemp)

In [18]:
playerLinks

['https://footystats.org/players/scotland/dominic-hyam',
 'https://footystats.org/players/england/sam-surridge',
 'https://footystats.org/players/scotland/ryan-christie',
 'https://footystats.org/players/england/ryan-nyambe',
 'https://footystats.org/players/england/karlan-ahearne-grant',
 'https://footystats.org/players/england/billy-mitchell',
 'https://footystats.org/players/england/ryan-giles',
 'https://footystats.org/players/england/adam-reach',
 'https://footystats.org/players/england/rayhaan-rahim-amari-tulloch',
 'https://footystats.org/players/england/romal-palmer',
 'https://footystats.org/players/england/joe-ward',
 'https://footystats.org/players/spain/alejandro-vallejo-minguez',
 'https://footystats.org/players/wales/benjamin-williams',
 'https://footystats.org/players/england/jordi-osei-tutu',
 'https://footystats.org/players/england/kwame-poku',
 'https://footystats.org/players/england/sam-baldock',
 'https://footystats.org/players/england/jahmari-clarke',
 'https://foo

In [ ]:
#We now have a list of player links from which to scrape their stats

In [214]:
L = len(playerLinks)
L

523

In [216]:
#Create Empty List to dump data into
data=[]

for i in range(0,L):
    url = playerLinks[i]
    driver = requests.get(url,headers=headers) 
    FinalSoup = BeautifulSoup(driver.content, 'html.parser')
    
    name = FinalSoup.find('p', attrs={'class':'col-lg-7 lh14e'})
    apps = FinalSoup.find('p', attrs={'class':'col-lg-1 ac semi-bold'})
    position = FinalSoup.find_all('p', attrs={'class':'col-lg-7 lh14e'})[2]
    club = FinalSoup.find_all('a', attrs={'class':'semi-bold'})[2]
    
    
    age = FinalSoup.find_all('p', attrs={'class':'col-lg-7 lh14e'})[3]
    
    for span in age('span',attrs={'class':'mild-small'}):
        span.decompose()
   
    country = FinalSoup.find_all('p', attrs={'class':'col-lg-7 lh14e'})[1]
    
    for span in country('span'):
        span.decompose()
    
    
    for n in name:
        data.append(n)
        
    for p in position:
        data.append(p)
        
    for ap in apps:
        data.append(ap)
        
    for cl in club:
        data.append(cl)
    
    for a in age:
        data.append(a)
        
    for c in country:
        data.append(c.text)
        

In [217]:
#Create DF of scraped data
ScrapedPlayers = pd.DataFrame()

ScrapedPlayers['Name']=data[0::6]
ScrapedPlayers['Position']=data[1::6]
ScrapedPlayers['Appearances']=data[2::6]
ScrapedPlayers['Club']=data[3::6]
ScrapedPlayers['Age']=data[4::6]
ScrapedPlayers['Country']=data[5::6]

In [218]:
ScrapedPlayers

,Name,Position,Appearances,Club,Age,Country
0,Dominic Hyam,Defender,9,Coventry City,25,Scotland
1,Sam Surridge,Forward,12,Stoke City,23,England
2,Ryan Christie,Midfielder,18,AFC Bournemouth,26,Scotland
3,Ryan Nyambe,Defender,7,Blackburn Rovers,23,England
4,Karlan Ahearne-Grant,Forward,9,West Bromwich Albion,23,England
...,...,...,...,...,...,...
518,Daniel Bentley,Goalkeeper,9,Bristol City,28,England
519,George Thomas,Midfielder,7,Queens Park Rangers,24,Wales
520,Rhys Norrington-Davies,Defender,7,Sheffield United,22,Wales
521,Anfernee Dijksteel,Midfielder,8,Middlesbrough,24,Netherlands


In [219]:
#Save DF to CSV file
ScrapedPlayers.to_csv('ChampionshipPlayersData.csv', encoding='utf-8', index=False, header=True)